# Chapter 3: Calculating the similarity between documents

Continue here - I need to make a decision about whether this is in the same chapter with tf-idf or not. Probably as otherwise the end of that chapter will involve manually calculating the tf-idf for every word, as well as being a bit boring and anti-climatic.

## Define the corpus

[I should put the frequencies of the original corpus here as well - this will just tie together this chapter to the previous chapter and orientate the reader to the chapter.]

Let's put together an example set, but this would also include all of the fairytales as well.

In [263]:
documents = [
    'The princess was clever',
    'The prince was handsome',
    'The prince loved the clever princess',
    '"Prince, handsome prince", said the princess',
]

## Create the index vocabulary

Index vocabulary is simply a numbered list of 'features' (i.e., terms) in our corpus. It includes (unless we explicitly define it) every word in the corpus, hence another reason to get rid of stop words early. These will also be the columns on the term-document matrix. 

In [264]:
index_vocabulary = {
    1: 'clever',
    2: 'handsome',
    3: 'loved',
    4: 'prince',
    5: 'princess',
    6: 'said'
}

In [15]:
index_vocabulary

{1: 'clever', 2: 'handsome', 3: 'loved', 4: 'prince', 5: 'princess', 6: 'said'}

## Turning documents into vectors

To turn a document into a vector, we simply need to count the number of time each term in the index vocabulary occurs in that document. Note that I need to strip out the punctuation and convert the words to lowercase. For the first document (sentence) in the corpus, we get:

In [16]:
import string

def count(document_list, term):
    doc = document_list.lower().translate(None, string.punctuation).split()
    match = [x for x in doc if x == term] 
    return len(match)

In [265]:
documents[0]

'The princess was clever'

In [18]:
m1 = [count(documents[0], x) for x in index_vocabulary.values()]
m1

[1, 0, 0, 0, 1, 0]

We can also represent this a bit more explicitly with a labelled dataframe.

In [19]:
from pandas import DataFrame
df = DataFrame(m1).T
df.columns = index_vocabulary.values()
df

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0


In [32]:
import numpy as np
tf_matrix = np.array(tuple([count(d, x) for x in index_vocabulary.values()] for d in documents))
tf_matrix

array([[1, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [0, 1, 0, 2, 1, 1]])

In [21]:
DataFrame(np.array(tuple([count(d, x) for x in index_vocabulary.values()] for d in documents)),
          columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0
1,0,1,0,1,0,0
2,1,0,1,1,1,0
3,0,1,0,2,1,1


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import operator
from pprint import pprint
vectorizer = CountVectorizer(stop_words = 'english')

vectorizer.fit_transform(documents)
sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))

[(u'clever', 0),
 (u'handsome', 1),
 (u'loved', 2),
 (u'prince', 3),
 (u'princess', 4),
 (u'said', 5)]

In [23]:
smatrix = vectorizer.transform(documents)
print smatrix

  (0, 0)	1
  (0, 4)	1
  (1, 1)	1
  (1, 3)	1
  (2, 0)	1
  (2, 2)	1
  (2, 3)	1
  (2, 4)	1
  (3, 1)	1
  (3, 3)	2
  (3, 4)	1
  (3, 5)	1


In [24]:
smatrix.todense()

matrix([[1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [1, 0, 1, 1, 1, 0],
        [0, 1, 0, 2, 1, 1]])

In [25]:
DataFrame(smatrix.todense(), columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0
1,0,1,0,1,0,0
2,1,0,1,1,1,0
3,0,1,0,2,1,1


## Term frequency-inverse document frequency

You might have already noticed a problem with using the raw frequency of words to characterise our documents. If we have a look at the term frequencies of our Grimm's fairytale corpus, we can see that our top 20 is dominated by words like 'say', 'go' and 'come'. These words are likely to occur with a high frequency in pretty much all of the tales, and therefore don't help us identify the unique topics of specific stories. What we want to do is weight the terms within a story so that terms that  occur many times within a small number of documents have the highest weight, and those that occur within all documents have the lowest weight. We can then determine which terms are most characteristic of specific documents.

We can do this with something called the term frequency-inverse document frequency, or tf-idf. This is simply a multiplication of the term frequencies in our document with something called the inverse document frequency (idf). Let's first cover how to calculate the idf before we move on to weighting the terms in our matrix.

In a general sense, the idf is simply some variant of dividing the total number of documents in a corpus by the number of documents the term of interest occurs in (or in other words, its document frequency). We'll use the [formula used by sklearn](http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting) in order to keep everything consistent in this chapter, which is calculated by:

$$idf(t) = log\frac{1 + n_d}{1 + df(d,t)} +1$$

In plain text, this means we add 1 to both the total number of documents ($n_d$) and the document frequency ($df(d,t)$) then divide this modified total number of documents by the modified document frequency. We then take the log of that, and add 1 to the whole thing. The reason for the additions of 1 to $n_d$ and $df(d,t)$ is to prevent divisions by zero, and the reason 1 is added to the final idf is so that terms that occur in every document don't get given a weighting of 0. Let's have a look at what the idf would be for our first term, 'clever':

In [270]:
# There are 4 documents
n = 4.0

# Clever occurs in 2 of them
df = 2.0

idf = math.log((1 + n) / (1 + df)) + 1
print("The idf of 'clever' is %0.2f.") % idf

The idf of 'clever' is 1.51.


Let's keep going and calculate the idf for each of the terms in our example corpus.

In [271]:
# Create function to clean up the string and strip out punctuation and case
def split_docs(document):
    doc = document.lower().translate(None, string.punctuation).split()
    return doc

# Calculate total number of documents in corpus
total_docs = len(documents)

# Calculate 1 + document frequency for each term in our vocabulary
term_dfs = [1.0 + len([count for count in [split_docs(tale).count(term) for tale in documents] 
                       if count != 0]) for term in index_vocabulary.values()]

# Calculate each idf
idfs = [math.log(y) + 1 for y in [(total_docs + 1) / x for x in term_dfs]]

# Print idfs with 
df = DataFrame(idfs).T
df.columns = index_vocabulary.values()
df

,clever,handsome,loved,prince,princess,said
0,1.510826,1.510826,1.916291,1.223144,1.223144,1.916291


Great! We now have all of our idfs. We can now move on to calculating the tf-idf. What we need to do is to our term frequencies and the inverse document frequencies. In applied terms, what this means is that we multiply each term frequency in a document by its associated idf. Let's have a look at a practical example for our fourth sentence. Our term frequency vector is:

In [272]:
tf_matrix[0]

array([1, 0, 0, 0, 1, 0])

And our idfs are:

In [273]:
idfs

[1.5108256237659907,
 1.5108256237659907,
 1.916290731874155,
 1.2231435513142097,
 1.2231435513142097,
 1.916290731874155]

So multiplying them together would look like:

$\begin{bmatrix}1\times1.51 & 0\times1.51 & 0\times1.92 & 0\times1.22 & 1\times1.22 & 0\times1.92 \end{bmatrix}$

which then simplifies to:

$\begin{bmatrix}1.51 & 0.00 & 0.00 & 0.00 & 1.22 & 0.00 \end{bmatrix}$

Let's now automate this for every document in our corpus and again pop it into a DataFrame with the vocabulary terms as columns.

In [274]:
tf_idfs = [y * idfs for y in tf_matrix]
DataFrame(tf_idfs, columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,1.510826,0.000000,0.000000,0.000000,1.223144,0.000000
1,0.000000,1.510826,0.000000,1.223144,0.000000,0.000000
2,1.510826,0.000000,1.916291,1.223144,1.223144,0.000000
3,0.000000,1.510826,0.000000,2.446287,1.223144,1.916291


## Vector normalisation

Raw term frequency can be misleading, as it weights towards very high occurrences of a term within a document, either because of keyword spamming or simply the length of the document. We can address this by normalising the vector. [Play with normalised and non-normalised vector for cosine similarity later and see how that looks.]

To normalise a vector, we simply need to change all of the values so that they add up to 1. To do this, we divide it by the length of the vector, or it's norm, and this norm can be calculated in a couple of ways. 

### L2-norm
In the diagram below, you can see that I've plotted the terms which have a non-zero frequency from the first matrix. You can I've plotted 'princess' on the x-axis, and 'clever' on the y-axis. As this document has a raw term frequency of 1 for each, we pop a point at (1, 1). If we draw a direct line between the origin and this point, how do we calculate the length of it? 

[graph 1 here - dot and line only]

Well, you might have noticed that when you draw lines between the origin and (0,1) and (1,0), this forms a right angle triangle. 

[graph 2 here - triangle]

We can now dust off our high school maths, and use Pythagoras's theorem to calculate the length of the vector. 

In [90]:
import math
l2_norm_0 = math.sqrt(sum([i ** 2 for i in tf_matrix[0]]))
print("The length of the vector is %0.3f.") % l2_norm_0

The length of the vector is 1.414.


This is known as the L2-norm, and can be generalised a vector of any length. For example, for the 4th document in our example set the L2-norm would be calculated as:

In [91]:
l2_norm_3 = math.sqrt(sum([i ** 2 for i in tf_matrix[3]]))
print("The length of the vector is %0.3f.") % l2_norm_3

The length of the vector is 2.646.


### L1-norm

[graph 3 here - dot and Manhattan distance only]
 
There is another way of calculating the distance between the origin and the point representing our term frequencies. Rather than taking the most direct path to the point (also known as the Euclidean distance), we can calculate what is called the Manhattan distance. This distance measure follows the most direct non-diagonal (like if a cab was trying to drive through Manhattan!), and is therefore simply the sum of all of its horizontal and vertical components. If we add up all of the lines in the graph above, you can see that it is 2, or in other words, just a sum of all of the term frequencies in the vector. Let's confirm this with our first vector.

In [77]:
l1_norm_0 = sum(tf_matrix[0])
print("The length of the vector is %d.") % l1_norm

The length of the vector is 2.


So how do we use these norms to normalise our vectors? Well, we simply divide our raw vector by our normalised vector. Easy peasy!

In the case of the L1-norm, you can see that the normalised vector adds up to 1.

In [80]:
tf_matrix[0] * 1.0 / l1_norm_0

array([ 0.5,  0. ,  0. ,  0. ,  0.5,  0. ])

In the case of the L2-norm, the *squared* results add to 1.

In [85]:
[tf**2 for tf in tf_matrix[0] * 1.0 / l2_norm_0]

[0.49999999999999989, 0.0, 0.0, 0.0, 0.49999999999999989, 0.0]

Let's now normalise our vectors. We can do so using the below list comprehensions, which divide each term in a vector by its L2-norm.

In [309]:
normed_freqs = [[term / math.sqrt(sum([num ** 2 for num in vec])) for term in vec] for vec in tf_idfs]
DataFrame(normed_freqs, columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,0.777221,0.000000,0.000000,0.000000,0.629228,0.000000
1,0.000000,0.777221,0.000000,0.629228,0.000000,0.000000
2,0.505100,0.000000,0.640655,0.408922,0.408922,0.000000
3,0.000000,0.412186,0.000000,0.667400,0.333700,0.522805


Let's just double-check that our vectors add to 1. We'll take the first vector, square each of the terms and add them. As all of the terms with a frequency of 0 won't contribute to the total, we'll just use the frequencies for 'clever' and 'princess':

In [319]:
normed_freqs[0][0]**2 + normed_freqs[0][4]**2

1.0

We can now add what we've learned about the tf-idf and vector normalisation together with the earlier work we did in Scikit-Learn. We'll first reuse the raw frequency matrix we created earlier in this chapter, and apply both the tf-idf and vector normalisation to it.

In [214]:
raw_freq_matrix = smatrix.todense()
raw_freq_matrix

matrix([[1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [1, 0, 1, 1, 1, 0],
        [0, 1, 0, 2, 1, 1]])

We'll use the `TfidfTransformer` method to transform our matrix. The first thing we want to do is to get the idf weights by running the `fit` function over our raw frequency matrix.

In [295]:
from sklearn.feature_extraction.text import TfidfTransformer

weights = TfidfTransformer(norm = 'l2')
weights.fit(raw_freq_matrix)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

You can see the idf weights by calling the `idf_` method:

In [296]:
weights.idf_

array([ 1.51082562,  1.51082562,  1.91629073,  1.22314355,  1.22314355,
        1.91629073])

Hopefully these look familiar! We've gotten the same result we did by calculating the results by hand earlier in this chapter. We can now use the `transform` method to both calculate the tf-idf and apply the L2-normalisation to each vector.

In [297]:
normed_matrix = weights.transform(raw_freq_matrix)
normed_matrix.todense()

matrix([[ 0.77722116,  0.        ,  0.        ,  0.        ,  0.62922751,
          0.        ],
        [ 0.        ,  0.77722116,  0.        ,  0.62922751,  0.        ,
          0.        ],
        [ 0.5051001 ,  0.        ,  0.64065543,  0.40892206,  0.40892206,
          0.        ],
        [ 0.        ,  0.41218562,  0.        ,  0.66739957,  0.33369979,
          0.5228052 ]])

And you can see we've gotten the exact same result as we did with our manual calculations. Now we can move onto something a bit more interesting - using these weights to work out what documents are similar.

## Cosine similarity

In order to work out how similar documents are, we can use something called the cosine similarity. Cosine, as in trigonometry? Yep, the same one. To make this more concrete, imagine we we had three documents, and we were trying to work out which ones are the most related based on the signal words 'clever' and 'princess'. The frequencies of these words in each of our imaginary documents is below:

In [366]:
DataFrame([['Document 1', 2, 7], ['Document 2', 2, 9], ['Document 3', 2, 1]], 
          columns = ['Document', 'clever', 'princess'])

,Document,clever,princess
0,Document 1,2,7
1,Document 2,2,9
2,Document 3,2,1


Given that documents 1 and 2 have very similar frequencies of both of our signal words, they are likely to be more closely related to each other than to the third document. We can make this a bit easier to see by plotting each of these points and draw a line between them and the origin, like we did earlier in the chapter. Remember that these lines are the representation of our document vectors when we only have 2 terms.

[graph 4 here - vectors]

You can see that the angle between the document 1 and document 2 vectors is much smaller than the angle between document 1 and document 3. But how much smaller? It might have occurred to you that, like when I was talking about vector normalisation, we can join two of the vectors and create a triangle. More specifically, we can make the last side perpendicular to the bottom vector to form a right angle triangle!

[graph 5 here - triangle 1]
[graph 6 here - triangle 2]

This should hopefully be looking like familiar territory. However, to quickly get those of you up to speed who haven't touched trigonometry since high school, the cosine is one of three ways to calculate the (non-perpendicular) angles on the inside of a right-angled triangle. It ranges from 1 to -1, with smaller angles having a value closer to 1. If you need a quick refresher on the cosine, [this page](https://www.khanacademy.org/math/trigonometry/trigonometry-right-triangles) has a good overview.

When we apply the cosine to vectors, we call it the cosine similarity. As tf-idf vectors normalise to 1, they can't be negative. As such, the cosine similarity in this context only ranges from 1 to 0, with documents that are very similar having a cosine similarity close to 1, while documents that are unrelated having a cosine similarity close to 0. 

Turns out we can calculate the cosine similarity simply using the information we have in our vectors. In order to get the cosine similarity, we first take the L2-norm of each vector and add them together. We then take the dot product of the two vectors - in other words, we multiply the first element in vector 1 by the first element in vector 2, repeat for all of the elements of each vector, and then sum the result. Finally, we divide the dot product of the vectors by the sum of the two normed vectors. And voila! We have the cosine similarity of our documents. 

Let's see how this looks with our example documents. 



## BREAK

Check - do you apply the term frequency to the raw frequency vector before you multiply it by the idf?
Steps:
* Calculate raw frequency vectors
* Apply sublinear term frequency
* Multiply by idf
* Apply L2-normalisation

In [321]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print tfidf_matrix.shape

(4, 6)


In [323]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)

array([[ 1.        ,  0.        ,  0.6498795 ,  0.20997309]])

In [327]:
print(tfidf_matrix.todense())

[[ 0.77722116  0.          0.          0.          0.62922751  0.        ]
 [ 0.          0.77722116  0.          0.62922751  0.          0.        ]
 [ 0.5051001   0.          0.64065543  0.40892206  0.40892206  0.        ]
 [ 0.          0.41218562  0.          0.66739957  0.33369979  0.5228052 ]]


In [340]:
d1_norm = math.sqrt(0.77722116**2 + 0.62922751**2)

In [341]:
d2_norm= math.sqrt(0.5051001**2 + 0.64065543**2 + 0.40892206**2 + 0.40892206**2)

In [339]:
d1_d2_dp = 0.77722116 * 0.5051001 + 0 * 0 + 0 * 0.64065543 + 0 * 0.40892206 + 0.62922751 * 0.40892206 + 0 * 0

In [343]:
cos_d1_d2 = d1_d2_dp / (d1_norm * d2_norm)
cos_d1_d2

0.6498795003666787

In [345]:
math.degrees(math.acos(cos_d1_d2))

49.467482665987525

In [370]:
m1 = [2, 7]
m2 = [2, 9]
m3 = [2, 1]

In [371]:
d1_d2_dp = (2 * 2 + 7 * 9)
d1_norm = math.sqrt(2**2 + 7**2)
d2_norm = math.sqrt(2**2 + 9**2)

In [372]:
cos_d1_d2 = d1_d2_dp / (d1_norm * d2_norm)
cos_d1_d2

0.9982226157912001

In [356]:
math.degrees(math.acos(cos_d1_d2))

3.4165881917711833

In [367]:
d1_d3_dp = (2 * 2 + 7 * 1)
d3_norm = math.sqrt(2**2 + 2**1)

In [368]:
cos_d1_d3 = d1_d3_dp / (d1_norm * d3_norm)
cos_d1_d3

0.6168493695012488

In [369]:
math.degrees(math.acos(cos_d1_d3))

51.91357798684324